In [1]:
# Cell 1: Import Libraries

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
import os

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries imported successfully!")

Libraries imported successfully!


## Step 1: Load Raw Datasets
Load air quality and physical features datasets for processing.

In [2]:
# Cell 2: Load Datasets

# File paths
aq_file = "data/processed/india_aq_transformed_last30days.csv"
pf_file = "data/processed/india_physical_features.csv"

print("="*70)
print("LOADING DATASETS")
print("="*70)

# Load Air Quality Data
if os.path.exists(aq_file):
    aq_df = pd.read_csv(aq_file)
    print(f"\n✅ Air Quality Dataset loaded: {len(aq_df):,} records")
    print(f"   Columns: {list(aq_df.columns)}")
else:
    print(f"❌ File not found: {aq_file}")
    aq_df = pd.DataFrame()

# Load Physical Features Data
if os.path.exists(pf_file):
    pf_df = pd.read_csv(pf_file)
    print(f"\n✅ Physical Features Dataset loaded: {len(pf_df)} locations")
    print(f"   Columns: {list(pf_df.columns)}")
else:
    print(f"❌ File not found: {pf_file}")
    pf_df = pd.DataFrame()

print("\n" + "-"*50)
print("Air Quality Data Preview:")
print("-"*50)
print(aq_df.head())

LOADING DATASETS

✅ Air Quality Dataset loaded: 109,501 records
   Columns: ['state', 'district', 'location_id', 'location_name', 'datetime_utc', 'datetime_local', 'latitude', 'longitude', 'pm25', 'pm10', 'no2', 'co', 'so2', 'o3', 'temperature', 'humidity', 'wind_speed', 'wind_direction']

✅ Physical Features Dataset loaded: 49 locations
   Columns: ['state', 'district', 'location_id', 'location_name', 'latitude', 'longitude', 'agricultural_area_sqm', 'agricultural_count', 'agricultural_distance_m', 'commercial_area_sqm', 'commercial_count', 'commercial_distance_m', 'dump_sites_area_sqm', 'dump_sites_count', 'dump_sites_distance_m', 'industrial_area_sqm', 'industrial_count', 'industrial_distance_m', 'power_plants_area_sqm', 'power_plants_count', 'power_plants_distance_m', 'residential_area_sqm', 'residential_count', 'residential_distance_m', 'roads_count', 'roads_distance_m', 'roads_total_length_m']

--------------------------------------------------
Air Quality Data Preview:
---------

## Step 2: Remove Duplicates and Invalid Records
Clean the dataset by removing duplicate entries and invalid records.

In [3]:
# Cell 3: Remove Duplicates and Invalid Records

print("="*70)
print("REMOVING DUPLICATES AND INVALID RECORDS")
print("="*70)

initial_count = len(aq_df)
print(f"\nInitial records: {initial_count:,}")

# Remove exact duplicates
aq_df = aq_df.drop_duplicates()
after_dup = len(aq_df)
print(f"After removing duplicates: {after_dup:,} (removed {initial_count - after_dup:,})")

# Remove records with invalid coordinates
aq_df = aq_df[(aq_df['latitude'].notna()) & (aq_df['longitude'].notna())]
aq_df = aq_df[(aq_df['latitude'] >= -90) & (aq_df['latitude'] <= 90)]
aq_df = aq_df[(aq_df['longitude'] >= -180) & (aq_df['longitude'] <= 180)]
after_coords = len(aq_df)
print(f"After removing invalid coordinates: {after_coords:,} (removed {after_dup - after_coords:,})")

# Remove records with missing timestamps
aq_df = aq_df[aq_df['datetime_utc'].notna()]
after_time = len(aq_df)
print(f"After removing missing timestamps: {after_time:,} (removed {after_coords - after_time:,})")

# Remove records where ALL pollutant values are missing
pollutant_cols = ['pm25', 'pm10', 'no2', 'co', 'so2', 'o3']
existing_pollutants = [col for col in pollutant_cols if col in aq_df.columns]
aq_df = aq_df.dropna(subset=existing_pollutants, how='all')
after_pollutants = len(aq_df)
print(f"After removing all-null pollutant records: {after_pollutants:,} (removed {after_time - after_pollutants:,})")

# Remove outliers (values outside 3 standard deviations)
for col in existing_pollutants:
    if col in aq_df.columns:
        mean_val = aq_df[col].mean()
        std_val = aq_df[col].std()
        lower_bound = mean_val - 3 * std_val
        upper_bound = mean_val + 3 * std_val
        # Only filter if values are unrealistic (negative or extremely high)
        aq_df = aq_df[(aq_df[col].isna()) | ((aq_df[col] >= 0) & (aq_df[col] <= upper_bound))]

final_count = len(aq_df)
print(f"After removing outliers: {final_count:,} (removed {after_pollutants - final_count:,})")

print(f"\n📊 Total records removed: {initial_count - final_count:,}")
print(f"📊 Final clean records: {final_count:,}")
print(f"📊 Data retention: {(final_count/initial_count)*100:.2f}%")

REMOVING DUPLICATES AND INVALID RECORDS

Initial records: 109,501
After removing duplicates: 109,501 (removed 0)
After removing invalid coordinates: 109,501 (removed 0)
After removing missing timestamps: 109,501 (removed 0)
After removing all-null pollutant records: 109,425 (removed 76)
After removing outliers: 106,369 (removed 3,056)

📊 Total records removed: 3,132
📊 Final clean records: 106,369
📊 Data retention: 97.14%


## Step 3: Handle Missing Values
Impute missing values using mean/median for pollutants and forward-fill for weather data.

In [4]:
# Cell 4: Handle Missing Values

print("="*70)
print("HANDLING MISSING VALUES")
print("="*70)

# Check missing values before imputation
print("\nMissing values BEFORE imputation:")
print("-"*50)
missing_before = aq_df.isnull().sum()
for col in aq_df.columns:
    missing = missing_before[col]
    pct = (missing / len(aq_df)) * 100
    if missing > 0:
        print(f"  {col:25s}: {missing:,} missing ({pct:.2f}%)")

# Define column categories
pollutant_cols = ['pm25', 'pm10', 'no2', 'co', 'so2', 'o3']
weather_cols = ['temperature', 'humidity', 'wind_speed', 'wind_direction']

# Impute pollutants with median (robust to outliers) grouped by location
print("\nImputing pollutant values with location-wise median...")
for col in pollutant_cols:
    if col in aq_df.columns:
        # Group by location and fill with median
        aq_df[col] = aq_df.groupby('location_id')[col].transform(
            lambda x: x.fillna(x.median())
        )
        # Fill remaining with global median
        aq_df[col] = aq_df[col].fillna(aq_df[col].median())

# Impute weather data with location-wise mean
print("Imputing weather values with location-wise mean...")
for col in weather_cols:
    if col in aq_df.columns:
        aq_df[col] = aq_df.groupby('location_id')[col].transform(
            lambda x: x.fillna(x.mean())
        )
        # Fill remaining with global mean
        aq_df[col] = aq_df[col].fillna(aq_df[col].mean())

# Check missing values after imputation
print("\nMissing values AFTER imputation:")
print("-"*50)
missing_after = aq_df.isnull().sum()
for col in aq_df.columns:
    missing = missing_after[col]
    if missing > 0:
        print(f"  {col:25s}: {missing:,} missing")

if missing_after.sum() == 0:
    print("  ✅ No missing values in numeric columns!")
else:
    print(f"  ⚠ Total remaining missing: {missing_after.sum():,}")

HANDLING MISSING VALUES

Missing values BEFORE imputation:
--------------------------------------------------
  pm25                     : 3,564 missing (3.35%)
  pm10                     : 8,245 missing (7.75%)
  no2                      : 3,493 missing (3.28%)
  co                       : 8,353 missing (7.85%)
  so2                      : 2,512 missing (2.36%)
  o3                       : 10,152 missing (9.54%)
  temperature              : 23,516 missing (22.11%)
  humidity                 : 15,882 missing (14.93%)
  wind_speed               : 20,271 missing (19.06%)
  wind_direction           : 17,946 missing (16.87%)

Imputing pollutant values with location-wise median...
Imputing weather values with location-wise mean...

Missing values AFTER imputation:
--------------------------------------------------
  ✅ No missing values in numeric columns!


## Step 4: Standardize Timestamps
Parse datetime strings and extract temporal components.

In [5]:
# Cell 5: Standardize Timestamps

print("="*70)
print("STANDARDIZING TIMESTAMPS")
print("="*70)

# Convert datetime_utc to datetime object
print("\nConverting datetime strings to datetime objects...")
aq_df['datetime_utc'] = pd.to_datetime(aq_df['datetime_utc'], utc=True)

# Convert to IST (Indian Standard Time) for local analysis
aq_df['datetime_ist'] = aq_df['datetime_utc'].dt.tz_convert('Asia/Kolkata')

# Extract date components
print("Extracting date components...")
aq_df['date'] = aq_df['datetime_ist'].dt.date
aq_df['year'] = aq_df['datetime_ist'].dt.year
aq_df['month'] = aq_df['datetime_ist'].dt.month
aq_df['day'] = aq_df['datetime_ist'].dt.day
aq_df['hour'] = aq_df['datetime_ist'].dt.hour
aq_df['day_of_week'] = aq_df['datetime_ist'].dt.dayofweek  # 0=Monday, 6=Sunday
aq_df['day_name'] = aq_df['datetime_ist'].dt.day_name()
aq_df['is_weekend'] = aq_df['day_of_week'].isin([5, 6]).astype(int)

# Round coordinates to standard precision
print("Standardizing GPS coordinates...")
aq_df['latitude'] = aq_df['latitude'].round(6)
aq_df['longitude'] = aq_df['longitude'].round(6)

print("\n✅ Timestamp standardization complete!")
print(f"\nDate range: {aq_df['date'].min()} to {aq_df['date'].max()}")
print(f"Hours covered: {sorted(aq_df['hour'].unique())}")

print("\n" + "-"*50)
print("Sample of standardized data:")
print("-"*50)
print(aq_df[['datetime_ist', 'date', 'hour', 'day_of_week', 'is_weekend']].head())

STANDARDIZING TIMESTAMPS

Converting datetime strings to datetime objects...
Extracting date components...
Standardizing GPS coordinates...

✅ Timestamp standardization complete!

Date range: 2025-11-08 to 2025-12-08
Hours covered: [np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11), np.int32(12), np.int32(13), np.int32(14), np.int32(15), np.int32(16), np.int32(17), np.int32(18), np.int32(19), np.int32(20), np.int32(21), np.int32(22), np.int32(23)]

--------------------------------------------------
Sample of standardized data:
--------------------------------------------------
               datetime_ist        date  hour  day_of_week  is_weekend
0 2025-11-08 15:15:00+05:30  2025-11-08    15            5           1
1 2025-11-08 15:30:00+05:30  2025-11-08    15            5           1
2 2025-11-08 15:45:00+05:30  2025-11-08    15            5           1
3 2025-11-08 16:00:00+05:

## Step 5: Derive Temporal Features
Create time-based features for pollution pattern analysis.

In [6]:
# Cell 6: Derive Temporal Features

print("="*70)
print("DERIVING TEMPORAL FEATURES")
print("="*70)

# Time of day categories
def get_time_of_day(hour):
    if 5 <= hour < 9:
        return 'early_morning'
    elif 9 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

print("\nCreating time of day categories...")
aq_df['time_of_day'] = aq_df['hour'].apply(get_time_of_day)

# Season based on month (for India)
def get_season(month):
    if month in [3, 4, 5]:
        return 'summer'
    elif month in [6, 7, 8, 9]:
        return 'monsoon'
    elif month in [10, 11]:
        return 'post_monsoon'
    else:  # 12, 1, 2
        return 'winter'

print("Creating season categories...")
aq_df['season'] = aq_df['month'].apply(get_season)

# Rush hour indicator (traffic-related pollution peaks)
print("Creating rush hour indicator...")
aq_df['is_rush_hour'] = aq_df['hour'].isin([8, 9, 10, 17, 18, 19, 20]).astype(int)

# Cyclical encoding for hour (captures circular nature of time)
print("Creating cyclical time encoding...")
aq_df['hour_sin'] = np.sin(2 * np.pi * aq_df['hour'] / 24)
aq_df['hour_cos'] = np.cos(2 * np.pi * aq_df['hour'] / 24)
aq_df['day_sin'] = np.sin(2 * np.pi * aq_df['day_of_week'] / 7)
aq_df['day_cos'] = np.cos(2 * np.pi * aq_df['day_of_week'] / 7)
aq_df['month_sin'] = np.sin(2 * np.pi * aq_df['month'] / 12)
aq_df['month_cos'] = np.cos(2 * np.pi * aq_df['month'] / 12)

print("\n✅ Temporal features created!")

print("\n" + "-"*50)
print("Temporal Feature Distribution:")
print("-"*50)
print(f"\nTime of Day:\n{aq_df['time_of_day'].value_counts()}")
print(f"\nSeason:\n{aq_df['season'].value_counts()}")
print(f"\nWeekend vs Weekday: Weekend={aq_df['is_weekend'].sum():,}, Weekday={len(aq_df) - aq_df['is_weekend'].sum():,}")

DERIVING TEMPORAL FEATURES

Creating time of day categories...
Creating season categories...
Creating rush hour indicator...
Creating cyclical time encoding...

✅ Temporal features created!

--------------------------------------------------
Temporal Feature Distribution:
--------------------------------------------------

Time of Day:
time_of_day
night            34780
afternoon        20510
early_morning    19480
evening          18251
morning          13348
Name: count, dtype: int64

Season:
season
post_monsoon    78351
winter          28018
Name: count, dtype: int64

Weekend vs Weekday: Weekend=34,339, Weekday=72,030


## Step 6: Merge with Physical Features
Combine air quality data with location-based physical features.

In [7]:
# Cell 7: Merge with Physical Features Dataset

print("="*70)
print("MERGING WITH PHYSICAL FEATURES")
print("="*70)

print(f"\nAir Quality records: {len(aq_df):,}")
print(f"Physical Features locations: {len(pf_df)}")

# Check common columns
print("\nPhysical Features columns:")
print(pf_df.columns.tolist())

# Merge on location_id
# Select relevant columns from physical features (excluding duplicate location info)
pf_cols_to_merge = [col for col in pf_df.columns if col not in 
                    ['state', 'district', 'location_name', 'latitude', 'longitude']]

print(f"\nMerging on 'location_id'...")
merged_df = pd.merge(
    aq_df,
    pf_df[pf_cols_to_merge],
    on='location_id',
    how='left'
)

print(f"\n✅ Merge complete!")
print(f"   Merged records: {len(merged_df):,}")
print(f"   Total columns: {len(merged_df.columns)}")

# Check for unmatched records
unmatched = merged_df[merged_df['roads_count'].isna()]['location_id'].nunique()
if unmatched > 0:
    print(f"   ⚠ Locations without physical features: {unmatched}")
else:
    print(f"   ✅ All locations matched with physical features!")

print("\n" + "-"*50)
print("Merged Dataset Preview:")
print("-"*50)
print(merged_df[['location_name', 'pm25', 'roads_count', 'industrial_count', 'agricultural_count']].head())

MERGING WITH PHYSICAL FEATURES

Air Quality records: 106,369
Physical Features locations: 49

Physical Features columns:
['state', 'district', 'location_id', 'location_name', 'latitude', 'longitude', 'agricultural_area_sqm', 'agricultural_count', 'agricultural_distance_m', 'commercial_area_sqm', 'commercial_count', 'commercial_distance_m', 'dump_sites_area_sqm', 'dump_sites_count', 'dump_sites_distance_m', 'industrial_area_sqm', 'industrial_count', 'industrial_distance_m', 'power_plants_area_sqm', 'power_plants_count', 'power_plants_distance_m', 'residential_area_sqm', 'residential_count', 'residential_distance_m', 'roads_count', 'roads_distance_m', 'roads_total_length_m']

Merging on 'location_id'...

✅ Merge complete!
   Merged records: 106,369
   Total columns: 57
   ✅ All locations matched with physical features!

--------------------------------------------------
Merged Dataset Preview:
--------------------------------------------------
                location_name  pm25  roads_c

## Step 7: Calculate Spatial Proximity Features
Create distance-based features for pollution source classification.

In [8]:
# Cell 8: Calculate Spatial Proximity Features

print("="*70)
print("CALCULATING SPATIAL PROXIMITY FEATURES")
print("="*70)

# Distance columns from physical features
distance_cols = [col for col in merged_df.columns if '_distance_m' in col]
print(f"\nDistance columns found: {distance_cols}")

# Create proximity categories based on distances
print("\nCreating proximity category features...")

def categorize_distance(distance, near=500, medium=1000):
    """Categorize distance as near/medium/far"""
    if pd.isna(distance):
        return 'unknown'
    elif distance <= near:
        return 'near'
    elif distance <= medium:
        return 'medium'
    else:
        return 'far'

# Create categorical proximity features
if 'roads_distance_m' in merged_df.columns:
    merged_df['road_proximity'] = merged_df['roads_distance_m'].apply(
        lambda x: categorize_distance(x, near=200, medium=500))
    print("  ✓ road_proximity created")

if 'industrial_distance_m' in merged_df.columns:
    merged_df['industrial_proximity'] = merged_df['industrial_distance_m'].apply(
        lambda x: categorize_distance(x, near=500, medium=1500))
    print("  ✓ industrial_proximity created")

if 'agricultural_distance_m' in merged_df.columns:
    merged_df['agricultural_proximity'] = merged_df['agricultural_distance_m'].apply(
        lambda x: categorize_distance(x, near=500, medium=1500))
    print("  ✓ agricultural_proximity created")

if 'dump_sites_distance_m' in merged_df.columns:
    merged_df['dump_proximity'] = merged_df['dump_sites_distance_m'].apply(
        lambda x: categorize_distance(x, near=500, medium=1500))
    print("  ✓ dump_proximity created")

# Create binary proximity flags (within 1km)
print("\nCreating binary proximity flags (within 1km)...")
if 'roads_distance_m' in merged_df.columns:
    merged_df['near_road'] = (merged_df['roads_distance_m'] <= 500).astype(int)
if 'industrial_distance_m' in merged_df.columns:
    merged_df['near_industry'] = (merged_df['industrial_distance_m'] <= 1000).astype(int)
if 'agricultural_distance_m' in merged_df.columns:
    merged_df['near_agriculture'] = (merged_df['agricultural_distance_m'] <= 1000).astype(int)
if 'dump_sites_distance_m' in merged_df.columns:
    merged_df['near_dump'] = (merged_df['dump_sites_distance_m'] <= 1000).astype(int)

print("\n✅ Spatial proximity features created!")

# Show proximity distribution
print("\n" + "-"*50)
print("Proximity Distribution:")
print("-"*50)
if 'road_proximity' in merged_df.columns:
    print(f"\nRoad Proximity:\n{merged_df['road_proximity'].value_counts()}")

CALCULATING SPATIAL PROXIMITY FEATURES

Distance columns found: ['agricultural_distance_m', 'commercial_distance_m', 'dump_sites_distance_m', 'industrial_distance_m', 'power_plants_distance_m', 'residential_distance_m', 'roads_distance_m']

Creating proximity category features...
  ✓ road_proximity created
  ✓ industrial_proximity created
  ✓ agricultural_proximity created
  ✓ dump_proximity created

Creating binary proximity flags (within 1km)...

✅ Spatial proximity features created!

--------------------------------------------------
Proximity Distribution:
--------------------------------------------------

Road Proximity:
road_proximity
near      78984
medium    18937
far        8448
Name: count, dtype: int64


## Step 8: Normalize Pollutant and Weather Values
Scale numeric features for consistent model input.

In [9]:
# Cell 9: Normalize Pollutant and Weather Values

print("="*70)
print("NORMALIZING NUMERIC FEATURES")
print("="*70)

# Define columns to normalize
pollutant_cols = ['pm25', 'pm10', 'no2', 'co', 'so2', 'o3']
weather_cols = ['temperature', 'humidity', 'wind_speed', 'wind_direction']

# Get existing columns
cols_to_normalize = [col for col in pollutant_cols + weather_cols if col in merged_df.columns]
print(f"\nColumns to normalize: {cols_to_normalize}")

# Create normalized versions using Min-Max scaling (0-1)
print("\nApplying Min-Max normalization (0-1 range)...")
scaler = MinMaxScaler()

for col in cols_to_normalize:
    col_data = merged_df[col].values.reshape(-1, 1)
    merged_df[f'{col}_normalized'] = scaler.fit_transform(col_data)
    print(f"  ✓ {col}_normalized created")

# Also create standardized versions (mean=0, std=1) for comparison
print("\nApplying Standard scaling (z-score)...")
std_scaler = StandardScaler()

for col in cols_to_normalize:
    col_data = merged_df[col].values.reshape(-1, 1)
    merged_df[f'{col}_scaled'] = std_scaler.fit_transform(col_data)
    print(f"  ✓ {col}_scaled created")

print("\n✅ Normalization complete!")

# Show sample of normalized data
print("\n" + "-"*50)
print("Sample Normalized Values:")
print("-"*50)
sample_cols = ['pm25', 'pm25_normalized', 'pm25_scaled'] if 'pm25' in merged_df.columns else []
if sample_cols:
    print(merged_df[sample_cols].describe())

NORMALIZING NUMERIC FEATURES

Columns to normalize: ['pm25', 'pm10', 'no2', 'co', 'so2', 'o3', 'temperature', 'humidity', 'wind_speed', 'wind_direction']

Applying Min-Max normalization (0-1 range)...
  ✓ pm25_normalized created
  ✓ pm10_normalized created
  ✓ no2_normalized created
  ✓ co_normalized created
  ✓ so2_normalized created
  ✓ o3_normalized created
  ✓ temperature_normalized created
  ✓ humidity_normalized created
  ✓ wind_speed_normalized created
  ✓ wind_direction_normalized created

Applying Standard scaling (z-score)...
  ✓ pm25_scaled created
  ✓ pm10_scaled created
  ✓ no2_scaled created
  ✓ co_scaled created
  ✓ so2_scaled created
  ✓ o3_scaled created
  ✓ temperature_scaled created
  ✓ humidity_scaled created
  ✓ wind_speed_scaled created
  ✓ wind_direction_scaled created

✅ Normalization complete!

--------------------------------------------------
Sample Normalized Values:
--------------------------------------------------
                pm25  pm25_normalized   p

## Step 9: Create Final Feature Set
Organize all features into a clean, unified DataFrame.

In [10]:
# Cell 10: Create Final Unified DataFrame

print("="*70)
print("CREATING FINAL UNIFIED DATASET")
print("="*70)

# Define column groups for final dataset
id_cols = ['state', 'district', 'location_id', 'location_name', 'latitude', 'longitude']
time_cols = ['datetime_utc', 'datetime_ist', 'date', 'year', 'month', 'day', 'hour', 
             'day_of_week', 'day_name', 'is_weekend', 'time_of_day', 'season', 'is_rush_hour']
cyclical_cols = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos']
pollutant_cols = ['pm25', 'pm10', 'no2', 'co', 'so2', 'o3']
weather_cols = ['temperature', 'humidity', 'wind_speed', 'wind_direction']

# Physical feature columns
physical_feature_cols = [col for col in merged_df.columns if any(x in col for x in 
    ['_count', '_distance_m', '_area_sqm', '_total_length_m'])]

# Proximity columns
proximity_cols = [col for col in merged_df.columns if '_proximity' in col or col.startswith('near_')]

# Normalized columns
normalized_cols = [col for col in merged_df.columns if '_normalized' in col or '_scaled' in col]

# Get all columns that exist
all_cols = (id_cols + time_cols + cyclical_cols + pollutant_cols + weather_cols + 
            physical_feature_cols + proximity_cols + normalized_cols)
final_cols = [col for col in all_cols if col in merged_df.columns]

# Create final DataFrame
final_df = merged_df[final_cols].copy()

# Sort by location and datetime
final_df = final_df.sort_values(['state', 'district', 'location_name', 'datetime_ist'])
final_df = final_df.reset_index(drop=True)

print(f"\n📊 Final Dataset Summary:")
print(f"   Total records: {len(final_df):,}")
print(f"   Total features: {len(final_df.columns)}")
print(f"   Unique locations: {final_df['location_id'].nunique()}")
print(f"   Date range: {final_df['date'].min()} to {final_df['date'].max()}")

print("\n" + "-"*50)
print("Feature Categories:")
print("-"*50)
print(f"  Identity columns: {len([c for c in id_cols if c in final_df.columns])}")
print(f"  Temporal columns: {len([c for c in time_cols + cyclical_cols if c in final_df.columns])}")
print(f"  Pollutant columns: {len([c for c in pollutant_cols if c in final_df.columns])}")
print(f"  Weather columns: {len([c for c in weather_cols if c in final_df.columns])}")
print(f"  Physical feature columns: {len([c for c in physical_feature_cols if c in final_df.columns])}")
print(f"  Proximity columns: {len([c for c in proximity_cols if c in final_df.columns])}")
print(f"  Normalized columns: {len([c for c in normalized_cols if c in final_df.columns])}")

print("\n" + "-"*50)
print("Final Dataset Preview:")
print("-"*50)
print(final_df.head())

CREATING FINAL UNIFIED DATASET

📊 Final Dataset Summary:
   Total records: 106,369
   Total features: 84
   Unique locations: 49
   Date range: 2025-11-08 to 2025-12-08

--------------------------------------------------
Feature Categories:
--------------------------------------------------
  Identity columns: 6
  Temporal columns: 19
  Pollutant columns: 6
  Weather columns: 4
  Physical feature columns: 21
  Proximity columns: 8
  Normalized columns: 20

--------------------------------------------------
Final Dataset Preview:
--------------------------------------------------
            state  district  location_id               location_name  \
0  Andhra Pradesh  Tirupati         5649  Tirumala, Tirupati - APPCB   
1  Andhra Pradesh  Tirupati         5649  Tirumala, Tirupati - APPCB   
2  Andhra Pradesh  Tirupati         5649  Tirumala, Tirupati - APPCB   
3  Andhra Pradesh  Tirupati         5649  Tirumala, Tirupati - APPCB   
4  Andhra Pradesh  Tirupati         5649  Tirumala, Ti

## Step 10: Save Cleaned Dataset
Export the final unified dataset for model training.

In [11]:
# Cell 11: Save Final Cleaned Dataset

print("="*70)
print("SAVING FINAL CLEANED DATASET")
print("="*70)

# Create output directory
output_dir = "data/cleaned"
os.makedirs(output_dir, exist_ok=True)

# Save main dataset
output_file = f"{output_dir}/india_aq_cleaned_features.csv"
final_df.to_csv(output_file, index=False)
print(f"\n✅ Main dataset saved: {output_file}")
print(f"   Size: {len(final_df):,} records x {len(final_df.columns)} columns")

# Save column metadata
print("\n" + "-"*50)
print("All Columns in Final Dataset:")
print("-"*50)
for i, col in enumerate(final_df.columns, 1):
    dtype = final_df[col].dtype
    non_null = final_df[col].notna().sum()
    print(f"  {i:2d}. {col:35s} ({dtype}) - {non_null:,} values")

# Save summary statistics
stats_file = f"{output_dir}/feature_statistics.csv"
final_df.describe().to_csv(stats_file)
print(f"\n✅ Statistics saved: {stats_file}")

print("\n" + "="*70)
print("DATA CLEANING AND FEATURE ENGINEERING COMPLETE!")
print("="*70)
print(f"\n📁 Output directory: {output_dir}")
print(f"📁 Main dataset: india_aq_cleaned_features.csv")
print(f"📁 Statistics: feature_statistics.csv")
print(f"\n🎉 Dataset is ready for EDA and model training!")

SAVING FINAL CLEANED DATASET

✅ Main dataset saved: data/cleaned/india_aq_cleaned_features.csv
   Size: 106,369 records x 84 columns

--------------------------------------------------
All Columns in Final Dataset:
--------------------------------------------------
   1. state                               (object) - 106,369 values
   2. district                            (object) - 106,369 values
   3. location_id                         (int64) - 106,369 values
   4. location_name                       (object) - 106,369 values
   5. latitude                            (float64) - 106,369 values
   6. longitude                           (float64) - 106,369 values
   7. datetime_utc                        (datetime64[ns, UTC]) - 106,369 values
   8. datetime_ist                        (datetime64[ns, Asia/Kolkata]) - 106,369 values
   9. date                                (object) - 106,369 values
  10. year                                (int32) - 106,369 values
  11. month        

## Summary

### Data Cleaning Operations:
1. ✅ Removed duplicate records
2. ✅ Removed invalid coordinates and timestamps
3. ✅ Handled missing values (median imputation for pollutants, mean for weather)
4. ✅ Removed outliers (>3 std deviations)

### Feature Engineering:
1. ✅ **Temporal Features**: hour, day_of_week, month, season, is_weekend, is_rush_hour
2. ✅ **Cyclical Encoding**: hour_sin/cos, day_sin/cos, month_sin/cos
3. ✅ **Spatial Features**: merged physical features (roads, industrial, agricultural, etc.)
4. ✅ **Proximity Features**: categorical and binary proximity indicators
5. ✅ **Normalized Features**: Min-Max (0-1) and Z-score scaling

### Output:
- `data/cleaned/india_aq_cleaned_features.csv` - Main cleaned dataset
- `data/cleaned/feature_statistics.csv` - Summary statistics